<a href="https://colab.research.google.com/github/danielepia/NLP-Twitter-API/blob/main/Twitter_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install advertools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 309 kB 6.7 MB/s 
     |████████████████████████████████| 264 kB 42.1 MB/s 
     |████████████████████████████████| 55 kB 2.4 MB/s 
     |████████████████████████████████| 251 kB 44.2 MB/s 
     |████████████████████████████████| 47 kB 3.3 MB/s 
     |████████████████████████████████| 4.0 MB 43.5 MB/s 
     |████████████████████████████████| 93 kB 1.6 MB/s 
     |████████████████████████████████| 3.1 MB 35.9 MB/s 
     |████████████████████████████████| 74 kB 1.4 MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-py3-none-any.whl size=11516 sha256=b74a1dc98be6ccefab758ea1114c7cd6961fa7ae0204b37e7e3846344b70734e
  Stored in directory: /root/.cache/pip/wheels/2d/18/21/3c6a732eaa69a339198e08bb63b7da2c45933a3428b29ec454
Successfully built PyDispatcher


In [ ]:
!pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 8.6 MB/s 


In [ ]:
import re
import tweepy
import advertools as adv
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

In [ ]:
class TwitterClient(object):

    def __init__(self):

        auth_params = {
            'app_key': 'xxxx',
            'app_secret': 'xxxx',
            'oauth_token': 'xxxx-xxxx',
            'oauth_token_secret': 'xxxx',
            }
  
        adv.twitter.set_auth_params(**auth_params)
  
    def clean_tweet(self, tweet):

        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split())
  
    def get_tweet_sentiment_comp(self, tweet):
        analyser = SentimentIntensityAnalyzer()
        analysis = analyser.polarity_scores(tweet)
        val = analysis['compound']
        return val

    def get_tweet_sentiment(self, val):
        if val > 0.05:
            return 'positive'
        elif val < -.05:
            return 'negative'
        else:
            return 'neutral'


    def get_tweets(self, user,field, count = 10):
        alerts_list = {user:field}
    
        for keys,values in alerts_list.items():
            df = adv.twitter.get_user_timeline(screen_name=keys,tweet_mode="extended")
            df = df[df['tweet_full_text'].str.contains(values,regex=True)]
 
        tweets = []

        for tweet in df['tweet_full_text']:
            parsed_tweet = {}
            parsed_tweet['text'] = tweet
            val = self.get_tweet_sentiment_comp(tweet)
            parsed_tweet['sentiment_comp'] = val
            parsed_tweet['sentiment'] = self.get_tweet_sentiment(val)
            tweets.append(parsed_tweet)
        return tweets


In [ ]:
def main():

    api = TwitterClient()

    tweets = api.get_tweets(user='FoxNews',field='', count = 10)
    tweets_df = pd.DataFrame(tweets).sort_values('sentiment_comp')

    print(tweets_df['sentiment'].value_counts()/tweets_df.shape[0])

    # printing first 5 positive tweets
    print("\n\nPositive tweets:")
    for i in tweets_df[tweets_df['sentiment']=='positive'].iloc[-5:]['text']:
      print(i)

    # printing first 5 negative tweets
    print("\n\nNegative tweets:")
    for i in tweets_df[tweets_df['sentiment']=='negative'].iloc[:5]['text']:
      print(i)  
  
if __name__ == "__main__":
    main()

2022-05-26 11:56:26,578 | INFO | twitter.py:238 | wrapper | get_user_timeline | Requesting: count=200, max_id=None, screen_name=FoxNews, tweet_mode=extended


negative    0.42
neutral     0.32
positive    0.26
Name: sentiment, dtype: float64


Positive tweets:
‘TOMORROW’S NOT PROMISED’: Xavier Lopez’s uncle describes his nephew to Fox News Digital, urges everyone to treasure their loved ones. https://t.co/rfMBLrKzNG https://t.co/8IW2H6FfJ3
US flag-carrying 'Old Glory Relay' for The World Games kicks off with veterans, military supporters https://t.co/V1f8A5a6zv
Queen Elizabeth was impressed by Kate Middleton’s adoration for Prince William despite royal status: author https://t.co/jyCwau9tQZ
RT @foxweather: WOW! ⚡ 

A line of strong thunderstorms put on quite a display of lightning for passengers flying over the Gulf Coast on Sa…
Mayorkas praises Border Patrol for response to Uvalde shooting, says heroism 'saved lives' https://t.co/ULaBKP5F1A


Negative tweets:
NYC subway shooting suspect charged with murder in 'unprovoked' death, uttered 'no phones' before attack: cops https://t.co/y9mAFZQl1D
TRAGEDY IN TEXAS: @GovAbbott addresses community 